# 最初から機械学習でやってみる

In [7]:
import os
import json
import time
import glob
from typing import TypeVar, Type, List, Optional
from dataclasses import dataclass
from pydantic import BaseModel
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from openai import AzureOpenAI
from tqdm import tqdm
from typing import List, Dict, Optional, Any, Tuple

In [8]:
csv_files = glob.glob('*.csv')
print(csv_files)

['extracted_500_first_20241130_検索結果1204.csv', 'pubmed_true_data.csv']


In [9]:
df = pd.read_csv(csv_files[0])
# Assuming your dataframe is named 'df'
df['included'] = df['notes'].astype(str).str.contains('Included')

print(df['included'].value_counts())
print(f'不一致の数: {df[df["included"] == True]["notes"].str.contains("Excluded").sum()}')
#予測の元となる変数

import json

def create_json_text(row):
    return json.dumps({
        'title': str(row['title']),
        'abstract': str(row['abstract'])
    })

df['tiab'] = df.apply(create_json_text, axis=1)

included
False    493
True       7
Name: count, dtype: int64
不一致の数: 0


In [14]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
import optuna
from lightgbm import LGBMClassifier

def prepare_text_data(df, text_column='tiab', label_column='included'):
    """テキストデータの前処理とTF-IDF変換"""
    texts = df[text_column].fillna('')

    vectorizer = TfidfVectorizer(
        max_features=10000,
        min_df=2,
        max_df=0.95,
        ngram_range=(1, 2)
    )

    X_vec = vectorizer.fit_transform(texts)
    y = df[label_column]

    class_weights = dict(zip(
        y.unique(),
        [1 / (len(y) * (y == label).mean()) for label in y.unique()]
    ))

    return X_vec, y, vectorizer, class_weights

def fbeta_score_custom(y_true, y_pred, beta=1):
    """カスタムF-betaスコアの計算"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    if precision == 0.0 and recall == 0.0:
        return 0.0
    fbeta = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
    return fbeta

def calculate_detailed_metrics(y_true, y_pred):
    """詳細な評価指標の計算と表示"""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0,1]).ravel()

    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    f1 = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0

    print("\n=== 詳細な評価指標 ===")
    print(f"感度 (Sensitivity/Recall): {sensitivity:.3f}")
    print(f"特異度 (Specificity): {specificity:.3f}")
    print(f"適合率 (Precision): {precision:.3f}")
    print(f"F1スコア: {f1:.3f}")

    print("\n=== 混同行列 ===")
    print("                  Predicted")
    print("                  Negative  Positive")
    print(f"Actual Negative    {tn:^8} {fp:^8}")
    print(f"      Positive    {fn:^8} {tp:^8}")

    return {
        'sensitivity': sensitivity,
        'specificity': specificity,
        'precision': precision,
        'f1': f1,
        'confusion_matrix': {'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp}
    }

def evaluate_model(X_vec, y, params, threshold, class_weights, beta=2):
    """モデルの評価（交差検証）"""
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    all_predictions = []
    all_true_values = []

    for train_idx, val_idx in skf.split(X_vec, y):
        X_tr = X_vec[train_idx]
        X_val = X_vec[val_idx]
        y_tr = y.iloc[train_idx]
        y_val = y.iloc[val_idx]

        model = LGBMClassifier(
            **params,
            random_state=42,
            class_weight=class_weights
        )

        model.fit(X_tr, y_tr)
        y_prob = model.predict_proba(X_val)[:, 1]
        y_pred = (y_prob >= threshold).astype(int)

        score = fbeta_score_custom(y_val, y_pred, beta=beta)
        scores.append(score)

        all_predictions.extend(y_pred)
        all_true_values.extend(y_val)

    if len(scores) == 5:
        print("\n=== 交差検証全体の評価 ===")
        metrics = calculate_detailed_metrics(all_true_values, all_predictions)
        print("\nClassification Report:")
        print(classification_report(all_true_values, all_predictions))

    return np.mean(scores)

def objective(trial, X_vec, y, class_weights, beta=2):
    """Optunaの目的関数"""
    params = {
        "num_leaves": trial.suggest_int("num_leaves", 8, 128),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0, log=True),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "verbosity": -1
    }
    threshold = trial.suggest_float("threshold", 0.1, 0.9)

    score = evaluate_model(X_vec, y, params, threshold, class_weights, beta=beta)
    return score

def train_final_model(df, beta_value=2, n_trials=50):
    """完全なトレーニングパイプライン"""
    # データの準備
    X_vec, y, vectorizer, class_weights = prepare_text_data(df)

    print(f"データセットの形状: {X_vec.shape}")
    print(f"クラスの分布:\n{y.value_counts(normalize=True)}")
    print(f"クラスの重み: {class_weights}")

    # ハイパーパラメータの最適化
    study = optuna.create_study(direction="maximize")
    study.optimize(
        lambda trial: objective(trial, X_vec, y, class_weights, beta=beta_value),
        n_trials=n_trials
    )

    best_params = study.best_trial.params
    threshold = best_params.pop("threshold")

    print("\nBest parameters:", best_params)
    print("Best threshold:", threshold)

    final_score = evaluate_model(X_vec, y, best_params, threshold, class_weights, beta=beta_value)
    print(f"\nFinal Mean F{beta_value} Score:", final_score)

    # 最終モデルの学習
    final_model = LGBMClassifier(
        **best_params,
        random_state=42,
        class_weight=class_weights
    )
    final_model.fit(X_vec, y)

    # 最終評価
    print("\n=== 最終モデルの評価（全データ） ===")
    y_prob = final_model.predict_proba(X_vec)[:, 1]
    y_pred = (y_prob >= threshold).astype(int)
    final_metrics = calculate_detailed_metrics(y, y_pred)

    return final_model, vectorizer, threshold, best_params

def predict_new_text(text, model, vectorizer, threshold):
    """新しいテキストの予測"""
    X_new = vectorizer.transform([text])
    prob = model.predict_proba(X_new)[0, 1]
    prediction = prob >= threshold
    return prediction, prob



In [15]:
# 使用例
if __name__ == "__main__":
    # データの読み込み
    #df = pd.read_csv('your_data.csv')  # tiab列とincluded列を含むデータ

    # モデルのトレーニング
    model, vectorizer, threshold, best_params = train_final_model(
        df,
        beta_value=4,  # より高い値で再現率を重視
        n_trials=50
    )


[I 2024-12-07 08:43:28,467] A new study created in memory with name: no-name-e1d0eb7e-c3f2-46ee-9206-27933ba53b0f


データセットの形状: (500, 10000)
クラスの分布:
included
False    0.986
True     0.014
Name: proportion, dtype: float64
クラスの重み: {np.False_: np.float64(0.002028397565922921), np.True_: np.float64(0.14285714285714285)}


[I 2024-12-07 08:43:31,733] Trial 0 finished with value: 0.1619047619047619 and parameters: {'num_leaves': 76, 'learning_rate': 0.0251301969404423, 'n_estimators': 221, 'reg_lambda': 0.004093897052729685, 'min_child_samples': 13, 'threshold': 0.11723547712465986}. Best is trial 0 with value: 0.1619047619047619.



=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 0.143
特異度 (Specificity): 0.980
適合率 (Precision): 0.091
F1スコア: 0.111

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative      483       10   
      Positive       6        1    

Classification Report:
              precision    recall  f1-score   support

       False       0.99      0.98      0.98       493
        True       0.09      0.14      0.11         7

    accuracy                           0.97       500
   macro avg       0.54      0.56      0.55       500
weighted avg       0.98      0.97      0.97       500



C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

...


=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 1.000
特異度 (Specificity): 0.000
適合率 (Precision): 0.014
F1スコア: 0.028

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative       0       493   
      Positive       0        7    

Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       493
        True       0.01      1.00      0.03         7

    accuracy                           0.01       500
   macro avg       0.01      0.50      0.01       500
weighted avg       0.00      0.01      0.00       500



C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i


=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 1.000
特異度 (Specificity): 0.000
適合率 (Precision): 0.014
F1スコア: 0.028

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative       0       493   
      Positive       0        7    

Classification Report:
              precision    recall  f1-score   support

       False       0.00      0.00      0.00       493
        True       0.01      1.00      0.03         7

    accuracy                           0.01       500
   macro avg       0.01      0.50      0.01       500
weighted avg       0.00      0.01      0.00       500



C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\youki\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i


=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 0.000
特異度 (Specificity): 1.000
適合率 (Precision): 0.000
F1スコア: 0.000

=== 混同行列 ===
                  Predicted
                  Negative  Positive
Actual Negative      493       0    
      Positive       7        0    

Classification Report:
              precision    recall  f1-score   support

       False       0.99      1.00      0.99       493
        True       0.00      0.00      0.00         7

    accuracy                           0.99       500
   macro avg       0.49      0.50      0.50       500
weighted avg       0.97      0.99      0.98       500


Best parameters: {'num_leaves': 89, 'learning_rate': 0.016608812657268206, 'n_estimators': 118, 'reg_lambda': 0.18836366455631795, 'min_child_samples': 79}
Best threshold: 0.4566199915963566

=== 交差検証全体の評価 ===

=== 詳細な評価指標 ===
感度 (Sensitivity/Recall): 0.714
特異度 (Specificity): 0.736
適合率 (Precision): 0.037
F1スコア: 0.070

=== 混同行列 ===
                  Predicted
       

In [16]:
import joblib


In [17]:
# 日時をstringsで
timestamp = time.strftime('%Y%m%d%H%M%S')

In [18]:

# モデルと関連オブジェクトの保存
model_data = {
    'model': model,
    'vectorizer': vectorizer,
    'threshold': threshold,
    'best_params': best_params
}
joblib.dump(model_data, f'{timestamp}_model_data.joblib')
print("\nモデルと関連オブジェクトを 'model_data.joblib' に保存しました。")


モデルと関連オブジェクトを 'model_data.joblib' に保存しました。


In [20]:
models = glob.glob('*.joblib')
print(models)

['model_data.joblib', '20241207084511_model_data.joblib']


In [21]:
# 保存したモデルデータをロード
model_data = joblib.load(models[1])

# 各オブジェクトの取り出し
loaded_model = model_data['model']
loaded_vectorizer = model_data['vectorizer']
loaded_threshold = model_data['threshold']
loaded_best_params = model_data['best_params']

print("モデルと関連オブジェクトをロードしました。")


モデルと関連オブジェクトをロードしました。


In [22]:
df2 = pd.read_csv(csv_files[1])
df2.columns = ["PMID", "title", "abstract"]
df2["tiab"] = df2.apply(create_json_text, axis=1)

In [23]:
df2.shape

(10, 4)

新しいデータの形状: (10, 10000)


In [36]:
def predict_and_summarize(df, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold):
    # "tiab"列の作成 (DataFrame dfに対して適用)
    df["tiab"] = df.apply(create_json_text, axis=1)

    # テキストデータの前処理とベクトル化
    X_new = loaded_vectorizer.transform(df["tiab"].fillna(''))

    print(f"新しいデータの形状: {X_new.shape}")

    # 予測確率の取得
    y_prob_new = loaded_model.predict_proba(X_new)[:, 1]

    # 閾値を適用してクラスラベルを決定
    y_pred_new = (y_prob_new >= loaded_threshold).astype(int)

    # 予測結果をデータフレームに追加
    df['prediction'] = y_pred_new
    df['probability'] = y_prob_new

    print("予測を実行し、結果をデータフレームに追加しました。")

    # 予測結果の表示
    print("\n=== 予測結果 ===")
    print(df[['tiab', 'prediction', 'probability']])

    # クラス分布の確認
    class_dist = df['prediction'].value_counts()
    print("\nクラスの分布:")
    print(class_dist)

    # 確率の統計情報
    prob_stats = df['probability'].describe()
    print("\n予測確率の統計情報:")
    print(prob_stats)

    # 必要に応じて、更新後のdfや統計情報を返す
    return df, class_dist, prob_stats


In [28]:
# 必要なオブジェクトを準備
# df2, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold を用意済みとする
updated_df, class_distribution, probability_stats = predict_and_summarize(
    df2, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold
)


新しいデータの形状: (10, 10000)
予測を実行し、結果をデータフレームに追加しました。

=== 予測結果 ===
                                                tiab  prediction  probability
0  {"title": "A multicenter, randomized, controll...           1     0.649690
1  {"title": "Transfusion requirements after head...           1     0.639201
2  {"title": "Restrictive vs Liberal Transfusion ...           1     0.566336
3  {"title": "Liberal or Restrictive Transfusion ...           1     0.573175
4  {"title": "Effect of Transfusion on Mortality ...           1     0.549861
5  {"title": "Transfusion-related risk of seconda...           1     0.555931
6  {"title": "Impact of Blood Product Transfusion...           1     0.604583
7  {"title": "Transfusion practice and blood stre...           1     0.576636
8  {"title": "Anemia, transfusions and hospital o...           1     0.633104
9  {"title": "Transfusion of platelets, but not o...           1     0.589637

クラスの分布:
prediction
1    10
Name: count, dtype: int64

予測確率の統計情報:
count    10.0

# 残っているファイルに対して予測を行い、Trueであったものだけを残す

In [33]:
df3 = pd.read_csv(r"searchdata\remaining_data_20241130.csv")

In [34]:
df3.head()

,key,title,authors,journal,volume,issue,pages,abstract,year,publisher,url,issn
0,cbe5dc78-690a-4f42-aa14-1fa928ee6f4a,Contemporary management of major haemorrhage i...,"[{'author': 'Maier, Cheryl L', 'author_abbrevi...",Intensive care medicine,50,3,319-331,Haemorrhagic shock is frequent in critical car...,2024,United States,https://doi.org/10.1007/s00134-023-07303-5,"{""pmid"": 38189930, ""electronic_issn"": ""1432-12..."
1,a6ee585f-1ba3-45a1-a701-9076213ea3e3,Pediatric Moderate and Severe Traumatic Brain ...,"[{'author': 'Ben Abdeljelil, Anis', 'author_ab...",Journal of neurotrauma,40,21-22,2270-2281,Traumatic brain injury (TBI) is the leading ca...,2023,United States,https://doi.org/10.1089/neu.2023.0149,"{""pmid"": 37341019, ""electronic_issn"": ""1557-90..."
2,fa95dc24-7daa-4768-8820-cf018f87ea59,Obstetric Disorders and Critical Illness.,"[{'author': 'Griffin, Kelly M', 'author_abbrev...",Clinics in chest medicine,43,3,471-488,"In this article, we discuss some of the more c...",2022,United States,https://doi.org/10.1016/j.ccm.2022.04.008,"{""pmid"": 36116815, ""electronic_issn"": ""1557-82..."
3,4c2a0dc7-e665-49bc-bf11-4208d2a5c141,How I diagnose and treat neonatal thrombocytop...,"[{'author': 'Stanworth, Simon J', 'author_abbr...",Blood,141,22,2685-2697,"Neonatal thrombocytopenia, defined as the pres...",2023,United States,https://doi.org/10.1182/blood.2022018017,"{""pmid"": 36787503, ""electronic_issn"": ""1528-00..."
4,426cc244-1011-4f64-935d-e1bada53efcf,Liberal or Restrictive Transfusion Strategy in...,"[{'author': 'Turgeon, Alexis F', 'author_abbre...",The New England journal of medicine,391,8,722-735,BACKGROUND: The effect of a liberal transfusio...,2024,United States,https://doi.org/10.1056/NEJMoa2404360,"{""pmid"": 38869931, ""electronic_issn"": ""1533-44..."


In [37]:
# 必要なオブジェクトを準備
# df2, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold を用意済みとする
updated_df, class_distribution, probability_stats = predict_and_summarize(
    df3, create_json_text, loaded_vectorizer, loaded_model, loaded_threshold
)


新しいデータの形状: (34900, 10000)
予測を実行し、結果をデータフレームに追加しました。

=== 予測結果 ===
                                                    tiab  prediction  \
0      {"title": "Contemporary management of major ha...           1   
1      {"title": "Pediatric Moderate and Severe Traum...           1   
2      {"title": "Obstetric Disorders and Critical Il...           0   
3      {"title": "How I diagnose and treat neonatal t...           1   
4      {"title": "Liberal or Restrictive Transfusion ...           1   
...                                                  ...         ...   
34895  {"title": "Age of Blood evalution.", "abstract...           1   
34896  {"title": "Autologous Umbilical Cord Blood Mon...           1   
34897  {"title": "Age of BLood Evaluation (ABLE) tria...           1   
34898  {"title": "Age of Red Blood Cells in Premature...           1   
34899  {"title": "The effect of transfusion on cerebr...           1   

       probability  
0         0.549861  
1         0.682654  
2     

In [38]:
print(df3.columns)

Index(['key', 'title', 'authors', 'journal', 'volume', 'issue', 'pages',
       'abstract', 'year', 'publisher', 'url', 'issn', 'tiab', 'prediction',
       'probability'],
      dtype='object')


In [39]:
df3["prediction"].value_counts()

prediction
0    23996
1    10904
Name: count, dtype: int64

In [40]:
df_export = df3[df3["prediction"] == 1]

In [41]:
df_export = df_export[['key', 'title', 'authors', 'journal', 'volume', 'issue', 'pages',
       'abstract', 'year', 'publisher', 'url', 'issn']]

In [42]:
df_export.shape

(10904, 12)

In [43]:
# df_exportをランダムに500とそれ以外に分割して、それぞれをCSVファイルとしてエクスポート, seed設定
df_export.sample(frac=1, random_state=42).iloc[:500].to_csv('exported_data500.csv', index=False)
df_export.sample(frac=1, random_state=42).iloc[500:].to_csv('remaining_data10404.csv', index=False)

